In [1]:
import pandas as pd

In [2]:
filename = 'data/officers_cohorts_data.csv'

In [102]:
date_cols = ['appointed_date', 'incident_date', 'birth_year']
dtypes = {'cohort': 'category',
          'gender': 'category',
          'race': 'category',
          'active': 'category',
          'disciplined_flag': 'category',
          'crid': 'str',
          'beat_id': 'str',
          'officer_id': 'category',
          'last_unit_id': 'category',
          'allegation_category_id': 'category'}


df = pd.read_csv(filename, index_col=0, parse_dates=date_cols, dtype=dtypes)
df = df.drop(['crew_id', 'community_id'], axis=1)
df.birth_year = df['birth_year'].dt.year

df['officer_age_incident'] = df['incident_date'].dt.year - df.birth_year
df['appointed_date'] = df['appointed_date'].dt.tz_localize('UTC')
df['officer_yof_incident'] = df['incident_date'] - df.appointed_date
df['days_on_force_incident'] = df['officer_yof_incident'].dt.days
df = df.drop(['officer_yof_incident'], axis=1)
df = df.drop(['point'], axis=1)
df = df.drop(['birth_year'], axis=1)

df.incident_date = df['incident_date'].dt.date
df.appointed_date = df['appointed_date'].dt.date
df = df.drop(['appointed_date'], axis=1)
df = df[~(df['officer_age_incident'] < 0)]
df = df[~(df['days_on_force_incident'] < 0)]
df = df.dropna(axis=0, subset=['incident_date'])

In [93]:
# produce a table for time series

print(df.head())

       officer_id cohort gender      race   active  complaint_percentile  \
#                                                                          
166306      22486      2      M     White      Yes               84.6051   
7638         1013      2      M  Hispanic      Yes               89.2094   
87381       11588      2      F     Black      Yes               60.3760   
243542      32166      1      M     Black  Unknown               99.8968   
163529      22087      2      M     Black       No               93.8191   

        civilian_allegation_percentile last_unit_id     crid incident_date  \
#                                                                            
166306                         81.8766           10   269165    2001-02-02   
7638                           91.4192           20   299906    2004-08-07   
87381                          57.7487          206  C255926    1999-08-17   
243542                         99.8206            7  C266045    2000-09-22   

In [104]:
acc_age = df.sample(n=20000, random_state=42)
acc_age = acc_age[['last_unit_id', 'officer_age_incident', 'coaccused_count', 'complaint_percentile']].copy()
acc_age['last_unit_id'] = 'Unit ' + df['last_unit_id'].astype(str)
acc_age.head()
acc_age.to_csv('scatter.csv', index=False)